In [3]:
import requests

# URLs for the files
urls = {
    "yolov3.weights": "https://pjreddie.com/media/files/yolov3.weights",
    "yolov3.cfg": "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg",
    "coco.names": "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"
}

# Download each file
for filename, url in urls.items():
    print(f"Downloading {filename}...")
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"{filename} downloaded successfully.")


yolov3.weights downloaded successfully.
yolov3.cfg downloaded successfully.
coco.names downloaded successfully.


In [ ]:
weight_path= r"C:\Car Detection App\yolov3.weights"
config_path= r"C:\Car Detection App\yolov3.cfg"
names_path= r"C:\Car Detection App\coco.names"

In [ ]:
# Loading YOLO v3
net= cv2.dnn.readNet(weight_path, config_path)
layer_names= net.getLayerNames()
output_layers= [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
# Load Coco Names
with open(names_path, "r") as f:
    classes= [line.strip() for line in f.readlines()]

In [ ]:
def car_detectn(image_path):
    
    img= cv2.imread(image_path)
    h_img, w_img, ch= img.shape()

    blob= cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop= False)
    net.setInput(blob)
    outs= net.forward(output_layers)

    class_ids= []
    confidences= []
    boxes= []
    for i in outs:
        for j in i:
            scores= detection[5:]
            class_id= np.argmax(scores)
            confidence= scores[class_id]
            if confidence > 0.5 and classes[class_id] == "car":
                
                # Object Parameters
                center_x= int(detection[0]*w_img)
                center_y= int(detection[1]*h_img)
                w= int(detection[2]*w_img)
                h= int(detection[3]*h_img)
                
                # Rectangular Coordinates
                x= int(center_x - w/2)
                y= int(center_y - h/2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes= cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h= boxes[i]
            label= str(classes[class_ids[i]])
            color= (0, 255, 0)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y-8), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 3)
    output_path= 'detected_car.jpg'
    cv2.imwrite(output_path, img)
    return output_path

    